<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_1105OpenAI_Gym_cartpole_rendering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # 2021年度駒澤大学心理学特講 IIIB

 ## OpenAI の gym 環境を用いた強化学習デモ

- filename: `2021_1105OpenAI_Gym_cartpole_rendering.ipynb`
- author: 浅川伸一
- data: 2021_1105

- [OpenAI の開発した強化学習環境 gym](http://gym.openai.com/)
- [starai による Google Colab 上で Gym をレンダリングする環境 ](https://star-ai.github.io/Rendering-OpenAi-Gym-in-Colaboratory/) 

上記 2 つの環境整備により実習が可能となります。すぐれた環境を提供してくれたすべての関係者に感謝いたします。

In [21]:
# 2022_1014
!pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 4.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 6.1 MB/s eta 0:00:0000:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110728 sha256=49328e4bc14360b17dfd3aafcba63805624230e5de04cba3e0da2ec6a748e1dc
  Stored in directory: /Users/_asakawa/Library/Caches/pip/wheels/29/15/e4/4f790bec6acd51a00b67e8ee1394f0bc6e0135c315f8ff399a
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [ ]:
# もしかしたら，このセルの実行は不要かも 2020-0715 メモ
# !apt-get update > /dev/null 2>&1
# !apt-get install cmake > /dev/null 2>&1
# !pip install --upgrade setuptools 2>&1
# !pip install ez_setup > /dev/null 2>&1

!pip install 'gym[atari]'
!pip install moviepy

# Gym とレンダリング環境が依存するライブラリのインストール


In [ ]:
#何が起こっているかを知りたい場合には下のコマンド中不等号以下をremove " > /dev/null 2>&1" を削除してください
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [1]:
# gym では，どのような環境が利用できるか表示します。
# 出力が長大になりますので，下までスクロールしてください。
from gym import envs
# for e in envs.registry.all():
#     print(e)
print(envs.registry.keys())

dict_keys(['ALE/Adventure-v5', 'ALE/Adventure-ram-v5', 'ALE/AirRaid-v5', 'ALE/AirRaid-ram-v5', 'ALE/Alien-v5', 'ALE/Alien-ram-v5', 'ALE/Amidar-v5', 'ALE/Amidar-ram-v5', 'ALE/Assault-v5', 'ALE/Assault-ram-v5', 'ALE/Asterix-v5', 'ALE/Asterix-ram-v5', 'ALE/Asteroids-v5', 'ALE/Asteroids-ram-v5', 'ALE/Atlantis-v5', 'ALE/Atlantis-ram-v5', 'ALE/Atlantis2-v5', 'ALE/Atlantis2-ram-v5', 'ALE/Backgammon-v5', 'ALE/Backgammon-ram-v5', 'ALE/BankHeist-v5', 'ALE/BankHeist-ram-v5', 'ALE/BasicMath-v5', 'ALE/BasicMath-ram-v5', 'ALE/BattleZone-v5', 'ALE/BattleZone-ram-v5', 'ALE/BeamRider-v5', 'ALE/BeamRider-ram-v5', 'ALE/Berzerk-v5', 'ALE/Berzerk-ram-v5', 'ALE/Blackjack-v5', 'ALE/Blackjack-ram-v5', 'ALE/Bowling-v5', 'ALE/Bowling-ram-v5', 'ALE/Boxing-v5', 'ALE/Boxing-ram-v5', 'ALE/Breakout-v5', 'ALE/Breakout-ram-v5', 'ALE/Carnival-v5', 'ALE/Carnival-ram-v5', 'ALE/Casino-v5', 'ALE/Casino-ram-v5', 'ALE/Centipede-v5', 'ALE/Centipede-ram-v5', 'ALE/ChopperCommand-v5', 'ALE/ChopperCommand-ram-v5', 'ALE/CrazyClimb

# 必要なヘルパ関数やライブラリをインポート


In [2]:
import gym
from gym import logger as gymlogger
#from gym.wrappers import Monitor
from gym.wrappers import RecordVideo

gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [3]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

process exited early. stderr:b"_XSERVTransmkdir: ERROR: euid != 0,directory /tmp/.X11-unix will not be created.\n_XSERVTransSocketUNIXCreateListener: mkdir(/tmp/.X11-unix) failed, errno = 2\n_XSERVTransMakeAllCOTSServerListeners: failed to create listener for local\n(EE) \nFatal server error:\n(EE) Cannot establish any listening sockets - Make sure an X server isn't already running(EE) \n"
start failed 1
process exited early. stderr:b"_XSERVTransmkdir: ERROR: euid != 0,directory /tmp/.X11-unix will not be created.\n_XSERVTransSocketUNIXCreateListener: mkdir(/tmp/.X11-unix) failed, errno = 2\n_XSERVTransMakeAllCOTSServerListeners: failed to create listener for local\n(EE) \nFatal server error:\n(EE) Cannot establish any listening sockets - Make sure an X server isn't already running(EE) \n"
start failed 2
process exited early. stderr:b"_XSERVTransmkdir: ERROR: euid != 0,directory /tmp/.X11-unix will not be created.\n_XSERVTransSocketUNIXCreateListener: mkdir(/tmp/.X11-unix) failed, errn

XStartError: No success after 10 retries. Last stderr: b"_XSERVTransmkdir: ERROR: euid != 0,directory /tmp/.X11-unix will not be created.\n_XSERVTransSocketUNIXCreateListener: mkdir(/tmp/.X11-unix) failed, errno = 2\n_XSERVTransMakeAllCOTSServerListeners: failed to create listener for local\n(EE) \nFatal server error:\n(EE) Cannot establish any listening sockets - Make sure an X server isn't already running(EE) \n"

In [7]:
"""
Gym 環境をビデオ撮影して表示するユーティリティ関数
ビデオを有効にするには "env = wrap_env(env) “ とするだけです。
"""

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
        loop controls style="height: 400px;">
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    

def wrap_env(env):
    #env = Monitor(env, './video', force=True)
    env = RecordVideo(env, './video')
    return env

# いろいろ遊んでみよう！

In [8]:
#env = wrap_env(gym.make("CartPole-v0"))
env = wrap_env(gym.make("CartPole-v1"))

# 下のパックマン環境は ROM が必要です
# env = wrap_env(gym.make("MsPacman-v0"))

In [9]:
# 上で選んだ環境の行動空間を表示してみましょう
print(env.action_space)

Discrete(2)


In [10]:
observation = env.reset()

while True:
  
    env.render()
    
    #ここにエージェントの動作を書く必要があります
    action = env.action_space.sample() 
    #observation, reward, done, info = env.step(action) 
    X = env.step(action) 
    observation, reward, done, info = X[0], X[1], X[2], X[3]
   
        
    if done:
        break;
            
env.close()
show_video()

Could not find video
